In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import random
import time
from network import *
from utility import *
from train import *
from config import *
import simpy
import logging

In [2]:
logger2 = None

In [3]:
#Create and configure logger 
def setup_logger (name, log_file):
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
    
    
    handler = logging.FileHandler(log_file , mode ='w')        
    handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    logger.addHandler(handler)

    return logger

In [4]:
%matplotlib notebook
plt.ion()

In [5]:
env = simpy.Environment()

In [6]:
N = Network(STATIONS_FILENAME, RAILWAY_FILENAME , env)

In [7]:
#utility function
def update_graph (env , interval):
    '''
    This function will simulate the whole process on a GUI
    This function is going to be used as the process
    @parameters
    env : simpy environment
    total_time : total time for the simulation
    '''
    fig = plt.figure(figsize = (6 , 6))
    ax = fig.add_subplot(111)
    fig.show()
    fig.canvas.draw()
    
    while True:
        #Draw graph
        
        N.draw_railway_network(ax  = ax , suppress_station_info= False , 
                               suppress_track_info= False , suppress_edge_label=False)

        #yield interval time
        yield env.timeout(interval)
        time.sleep(0.5)
        
        #log info
        #logger.info("Time : {} Updating Graph".format(env.now))
        plt.title("Time {}".format(env.now))
        
        #Paint the canvas
        fig.canvas.draw()

In [8]:
def read_trains(filename):
    
    '''
    Reads the trains and put them in the list from the given filename
    '''
    trains = []
    with open(filename) as f:
        
        while True:
            line = f.readline().strip().split()
            
            if len(line) == 4:
                name , speed , priority, route_len = line[0] , int(line[1]) , int(line[2]) , int(line[3])

                #Read the route
                route = []
                for _ in range(route_len):
                    line = f.readline().strip().split()
                    route.append((line[0] , int(line[1]) , int(line[2]) ))
                
                line = f.readline().strip().split()
                #Create train instance and append it to the list
                log_filename = "Logs/" + name + ".log"
                
                logger = setup_logger(name , log_filename)
                train = Train (name , speed , priority , route , env , N , logger)
                trains.append(train)
            else:
                break
    return trains
        

In [9]:
def deadlock_detection_process (env):
    '''
    For creating deadlock detection process
    '''
    global DEADLOCK
    while True:
        
        #Invoke this deadlock process after every 20 iterations
        
        yield env.timeout(20)
        logger.info("Time : {} Checking Deadlock".format(env.now))
        DEADLOCK , _ = deadlock_detection(N , trains)
        


In [10]:
def run_simulation (trains , total_time):
    
    '''
    Takes the train and total_time for simulation and run it    
    '''
    for t in trains:
        proc = env.process(t.move_train_simulate(env))

    sim_proc = env.process(update_graph(env , 1))
    env.run(until =total_time)

In [11]:
trains = read_trains(TRAINS_FILENAME)

In [12]:
name_train_map = {}                #Map from train name to train object; note key can be changed to train_id 
for t in trains:
    name_train_map[t.name] = t

In [13]:
logger = setup_logger("main_log_file" ,"Logs/log.log")

In [14]:
def choose_action_simulate (env):
    
    '''
    This function will choose the action for the trains that need action at the
    particular simulation time
    
    This will not check if the move is valid or not.
    In case of invalid moves, train will wait till the resource is freed.
    '''
    
    global TRAINS_NEEDING_ACTION
    actions = ['move' , 'wait']
    
    while True :
        #This line is extremely important
        #As this line will wait for all the events upto this current simulation point to complete first
        #and then this will execute
        for _ in range(4*len(trains)):
            yield env.timeout(0)
        
        #Check if the trains need action
        if (len(TRAINS_NEEDING_ACTION) == 0):
            yield env.timeout(1)
        
        else:
            
            #Pick the train that needs the action mose
            #TODO : choose the train that needs action based on the heuristic
            time , name = TRAINS_NEEDING_ACTION[0]
            t = name_train_map[name]
        
            #Pick the action
            action = np.random.choice(actions , 1 , p = [0.5 , 0.5])          
        
            #Create the process that completes the action
            env.process(t.act_simulate(env , action))
            
            #Remove the name of the train from the list
            TRAINS_NEEDING_ACTION.remove((time,  name) )
            
            

In [15]:
def choose_action (env):
    
    '''
    This function will choose the action for the trains that need action at the
    particular simulation time
    
    This will also check if the move is valid or not.
    '''
    
    global TRAINS_NEEDING_ACTION
    actions = ['move' , 'wait']
    
    while True :
        #This line is extremely important
        #As this line will wait for all the events upto this current simulation point to complete first
        #and then this will execute
        for _ in range(4*len(trains)):
            yield env.timeout(0)
        
        #Check if the trains need action
        if (len(TRAINS_NEEDING_ACTION) == 0):
            yield env.timeout(1)
        
        else:
            
            #Pick the train that needs the action most
            #TODO : choose the train that needs action based on the heuristic
            time ,name = pick_most_suitable_action(name_train_map , N , env)
            t = name_train_map[name]
        
            #Pick the action
            action = np.random.choice(actions , 1 , p = [1 , 0])          
        
            #if the action is to move, then check if move is valid or not
            #if not : then implement the wait move
            if not (t.is_move_valid(env)):
                action = 'wait'
                
            
            
            #Create the process that completes the action
            env.process(t.act_simulate(env , action))
            
            #Remove the name of the train from the list
            TRAINS_NEEDING_ACTION.remove((time,  name) )
            
            

In [16]:
# sim_proc = env.process(update_graph(env , 1))
dead_proc = env.process(deadlock_detection_process(env))

for t in trains:
    proc = env.process(t.put_train_on_track(env))
    
proc_act = env.process(choose_action(env))

In [17]:
for _ in range(TOTAL_SIMULATION_TIME):
    
    if (DEADLOCK== False):
        logger.warning("Time : {} Terminating Simulation. Deadlock detected".format(env.now))
        print("Terminating Simulation : Deadlock detected")
        break
    
    
    CURRENT_SIMULATION_TIME += 1
    
    
    #Run the simulation
    env.run(until = CURRENT_SIMULATION_TIME)
    

[('Train001', 'Alpha', 0, 1), ('Train002', 'Alpha', 0, 1), ('Train003', 'Alpha', 0, 1), ('Train004', 'Alpha', 0, 1), ('Train011', 'Echo', 0, 1), ('Train012', 'Echo', 0, 1), ('Train013', 'Echo', 0, 1), ('Train014', 'Echo', 0, 1)]


KeyError: '_'

In [ ]:
G,train_nodes , station_nodes , track_nodes = create_resource_usage_graph(trains , N)

In [ ]:
fig = plt.figure(figsize = (6 , 6))
ax = fig.add_subplot(111)
draw_network_usage_graph(G,train_nodes , station_nodes , track_nodes, N,ax)

In [ ]:
# TODO
'''
1. Create a simple train class with all the parameters                                                Done
2. Implement the resource facility with the track and the station                                     Done
3. Run the single train on the track (without getting worried wether the resource is free or not)     Done  
4. Simulate whole thing on graph (single train only)                                                  Done
5. Run multiple instances of the train without worrying about the deadlock                            Done
   Create action for each train                                                                       Done
6. Try to simulate as you want to do in the project i.e. take action from the user.                   Done
   Also able to order the train for which to take the action at a particular time 
   
   
   
7. Try to create deadlock with the trains                                                             Done 
8. Create the graph for resource usage.                                                               Done  
9. Use the standard deadlock detection algorithm for the detection of the algorithm                   Done 
   Implement the Bank's algorithm for deadlock detection.
'''

In [ ]:
'''
List of reasons of creating events
1. If a train is standing at a station, the event processing time            Can be done by implementing timeout if train arrive early before depart time
    corresponds to the earliest time at which the train can depart,          occupancy of next track.
    as defined by its minimum halt time at the station and by any           Done
    departure time constraints enforced for passenger convenience.

2. `If it is running between two stations, the event processing time         Already done   
    corresponds to the earliest time at which it can arrive at the          occupancy of next station if move.
    next station, as defined by the length of the track and the train       Done
    running speed.

3. If the train is yet to start, the event processing                       Done
    time is the time at which it is expected at the starting station.       
'''

In [ ]:
#TODO
'''
Create the actions properly and implement deadlock Avoidance heuristic

1. Code the proper list of events. All the three listed above.                                      Done
2. Create the proper time table with arrival and departure time of each train at each station.      Done
3. Create separate log file for each train.                                                         Done
3. Run simulation with the first toy environment (as in the paper).                                 Done
4. Label the current set of implementation under Simulation Phase.                                  Done
5. Implement the actions properly without actually waiting for the resource.                        Done
6. Understand Heuristic that can be used to avoid deadlock.                                         Done
7. Move that train that occupies the most congested resource first and then other.                Not Done
   If tie break it using priority of each train.
   
   
8. Implement the state vector when action need to be taken.                                       Note Done

'''

In [ ]:
avail = np.array([0,0,0])  
  
# Maximum R that can be allocated  
# to processes  
maxm = np.array([[0, 1, 0], [4, 0, 2], 
        [3, 0, 4], [3, 1, 1], 
        [0, 0, 4]]) 

# Resources allocated to processes  
allot = np.array([[0, 1, 0], [2, 0, 0], 
         [3, 0, 3], [2, 1, 1], 
         [0, 0, 2]])  

# Check system is in safe state or not  
isSafe(avail, maxm, allot)  

In [ ]:
deadlock_detection (N , trains)

In [ ]:
def choose_action_one_train (env, train):
    '''
    After each unit time: this function will check if the train need the action or not
    If yes : this will take the action for the train and check again if it needs action
    This keeps on happening
    '''
    
    global TRAINS_NEEDING_ACTION
    actions = ['move' , 'wait']
    while True:
        
        #keep on taking the action for the train till it is not present in TRAINS_NEEDING_ACTION
        while (train.name in TRAINS_NEEDING_ACTION):
            action = np.random.choice(actions , 1 , p = [1 , 0])

            #Take the action
            yield env.process(t.act(env , action))

            #remove the train from the list
            TRAINS_NEEDING_ACTION.remove(train.name)
        
        yield env.timeout(1)
        

In [ ]:
def choose_action_temp (env):
    #Process the action for the trains in the list    
    #randomly take action for each train waiting for action 
    #We can also order the train for which we are taking the action depending on the situation
    global TRAINS_NEEDING_ACTION
    actions = ['move' , 'wait']
    
    temp_trains_needing_action = TRAINS_NEEDING_ACTION
    
    for name in temp_trains_needing_action:
        t = name_train_map[name]
        
        action = np.random.choice(actions , 1 , p = [1 , 0])          #move with 50% probability
        
        #Take the action
        yield env.process(t.act(env , action))
        
        #remove the train from the list
        TRAINS_NEEDING_ACTION.remove(name)
        
        

In [ ]:
def choose_action_process(env):
    #Choose action
    while True:
        while (not len(TRAINS_NEEDING_ACTION) == 0 ):
            yield env.process(choose_action(env))
    
        yield env.timeout(1)